In [14]:
%%bash
mkdir -p sample
echo "" > sample/__init__.py

In [29]:
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


def main():
    tf.compat.v1.experimental.output_all_intermediates(True)
    seq_data = keras.Input(shape=(None, 1),  name="seq_data")
    seq_lengths = keras.Input(shape=(), name="seq_lengths", dtype=tf.int32)

    mask = keras.layers.Lambda(lambda x: tf.sequence_mask(x))(seq_lengths)
    conv = layers.Conv1D(32, 3, strides=1, padding='same', activation='relu')(seq_data)
    #rnn = layers.Bidirectional(layers.GRU(32, return_sequences=True))(conv,mask=mask)
    rnn = layers.Bidirectional(layers.GRU(32, return_sequences=True))(conv)
    dense = layers.Dense(5, name="signal_mask")(rnn)
    model = keras.Model(inputs=[seq_data, seq_lengths], outputs=[dense])
    print(model.input_shape, model.output_shape)

    model.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.001),
        loss=tf.keras.backend.sparse_categorical_crossentropy
    )

    batch_size = 1
    time_step = 1000
    signal_length = 1000
    sequences = np.random.rand(batch_size, time_step, 1)
    seq_lengths = np.array([signal_length])
    signal_mask = np.random.randint(5, size=(1, time_step))
    #def map_fn(a,b,c):
    #    return ({"seq_data":a,"seq_lengths":b}, {"signal_mask":c})
    #dataset = tf.data.Dataset.from_tensor_slices(
    #    (sequences, seq_lengths,  signal_mask)).repeat(100).batch(1).map(map_fn)

    dataset = tf.data.Dataset.from_tensor_slices(((sequences, seq_lengths),  signal_mask)).repeat(100).batch(1)

    model.fit(dataset, epochs=1)
    
main()

[(None, None, 1), (None,)] (None, None, 5)
Train on 100 steps
100/100 [==============================] - 152s 2s/step - loss: 1.6452


In [21]:
%%writefile sample/model.py

import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


def main():
    mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
                        
    with mirrored_strategy.scope():
        seq_data = keras.Input(shape=(None, 1),  name="seq_data")
        seq_lengths = keras.Input(shape=(), name="seq_lengths", dtype=tf.int32)

        mask = keras.layers.Lambda(lambda x: tf.sequence_mask(x))(seq_lengths)
        conv = layers.Conv1D(32, 3, strides=1, padding='same', activation='relu')(seq_data)
        #rnn = layers.Bidirectional(layers.GRU(32, return_sequences=True))(conv,mask=mask)
        rnn = layers.Bidirectional(layers.GRU(32, return_sequences=True))(conv)
        dense = layers.Dense(5, name="signal_mask")(rnn)
        model = keras.Model(inputs=[seq_data, seq_lengths], outputs=[dense])

        model.compile(optimizer=tf.keras.optimizers.Adam(
            learning_rate=0.001),
            loss=tf.keras.backend.sparse_categorical_crossentropy
        )

    batch_size = 1
    time_step = 1000
    signal_length = 1000
    sequences = np.random.rand(batch_size, time_step, 1)
    seq_lengths = np.array([signal_length])
    signal_mask = np.random.randint(5, size=(1, time_step))

    dataset = tf.data.Dataset.from_tensor_slices(
        (sequences, seq_lengths,  signal_mask)).repeat(100).batch(1)\
        .map(lambda a,b,c: ({"seq_data":a,"seq_lengths":b}, {"signal_mask":c}))

    model.fit(dataset, epochs=1)
    
main()

Writing sample/model.py


FileNotFoundError: [Errno 2] No such file or directory: 'sample/model.py'

In [36]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  # Configure a master worker with 1 with K80 GPUs
  masterType: n1-highcpu-16
  masterConfig:
    acceleratorConfig:
      count: 2
      type: NVIDIA_TESLA_K80
  # Configure 1 worker(s), each with 2 K80
  workerCount: 1
  workerType: n1-highcpu-16
  workerConfig:
    acceleratorConfig:
      count: 2
      type: NVIDIA_TESLA_K80

Overwriting config.yaml


In [37]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="multi_cpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://gp-ex \
  --package-path=sample \
  --module-name=sample.model \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-west1 \
  --config config.yaml
  

jobId: multi_cpu_fashion_minst_20200730_191020
state: QUEUED


Job [multi_cpu_fashion_minst_20200730_191020] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe multi_cpu_fashion_minst_20200730_191020

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs multi_cpu_fashion_minst_20200730_191020


In [24]:
import tensorflow
print(tensorflow.__version__)

2.2.0-dlenv


In [3]:
x = keras.Input(shape=(2,))

In [4]:
x

<tf.Tensor 'input_1:0' shape=(None, 2) dtype=float32>

In [6]:
y=2*x

In [7]:
y

<tf.Tensor 'mul:0' shape=(None, 2) dtype=float32>

In [8]:
y(2)

TypeError: 'Tensor' object is not callable